In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
from dotenv import load_dotenv
# from langchain.document_loaders import JSONLoader

In [3]:
import os
import openai
from dotenv import load_dotenv

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = ""
os.environ["OPENAI_API_KEY"] = ""
os.environ["DEFAULT_EMBED_BATCH_SIZE"] = "10"

In [4]:
from langchain import OpenAI, PromptTemplate, LLMChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.mapreduce import MapReduceChain
from langchain.prompts import PromptTemplate

In [14]:
llm = OpenAI(engine="davinci",temperature=0)
text_splitter = CharacterTextSplitter()

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [15]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("data/SparkOfGPT-4.pdf")
pages = loader.load()

In [20]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(pages, disallowed_special=())

TypeError: TextSplitter.split_documents() got an unexpected keyword argument 'disallowed_special'

In [51]:
# from langchain.document_loaders import JSONLoader
# loader = JSONLoader("data/MedicalRecords.json", jq_schema='.key[].text')
# pages = loader.load_and_split()

In [17]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

In [18]:
faiss_index = FAISS.from_documents(docs, OpenAIEmbeddings(chunk_size=1))
docs = faiss_index.similarity_search("what is GPT-4?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content)

ValueError: Encountered text corresponding to disallowed special token '<|endofprompt|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endofprompt|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endofprompt|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


In [46]:
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [47]:

from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(model="gpt-35-turbo")

In [48]:
from langchain.chains.summarize import load_summarize_chain

In [49]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(temperature=0.7, engine="gpt-4")
# llm=ChatOpenAI(temperature=0.7, engine="gpt-4", max_tokens=1000)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [50]:
prompt_template = """Write a concise summary of the following:


{text}

"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
# chain = load_summarize_chain(llm, chain_type="map_reduce", return_intermediate_steps=False, map_prompt=PROMPT, combine_prompt=PROMPT)
chain = load_summarize_chain(llm, chain_type="refine", return_intermediate_steps=True)
chain({"input_documents": docs}, return_only_outputs=True)

{'intermediate_steps': ['This paper explores the potential of large language models (LLMs) in improving personalized user experiences on recommendation platforms. The authors introduce a framework for personalized extraction of "interest journeys," or persistent user interests, and summarize these journeys using LLMs with techniques like few-shot prompting, prompt-tuning, and fine-tuning. The results demonstrate the potential of LLMs in providing deeper, more interpretable, and controllable user understanding, paving the way for new user experiences on recommendation platforms that are journey-aware, assistive, and enable frictionless conversation.',
  'This paper explores the potential of large language models (LLMs) in improving personalized user experiences on recommendation platforms. The authors introduce a framework for personalized extraction of "interest journeys," or persistent user interests, needs, and goals, by leveraging personalized clustering from user interaction logs. 